<a href="https://colab.research.google.com/github/strawndri/corretor_ortografico-nlp/blob/main/Colaboratory/Corretor_ortografico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP - Processamento de Linguagem Natural

A **Processamento de Linguagem Natural (NLP, em inglês)** está relacionada à Inteligência Antificial. Este mecanismo, portanto, atua na conexão entre o ser humano e a máquina.

A interpretação de textos realizada por pessoas, é de certa forma, simples. No entanto, quando este papel cabe à máquina, a situação torna-se mais complicada, uma vez que ela precisa "compreender" inúmeros peculiaridades linguísticas, como coensão, coêrencia, morfologia, semântica, sintaxe, etc.

Alguns exemplos de atividades comuns que utlizam o NLP são: tradução automática, corretor ortográfico, autocomplete (realizado pelo Google, por exemplo), chatbots...

# Importações

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Importando um Corpus Textual
Obs: Corpus significa, em NLP, uma coleção de documentos

In [4]:
with open('artigos.txt', 'r') as f:
  artigos = f.read()

print(artigos[:500])




imagem 

Temos a seguinte classe que representa um usuário no nosso sistema:

java

Para salvar um novo usuário, várias validações são feitas, como por exemplo: Ver se o nome só contém letras, [**o CPF só números**] e ver se o usuário possui no mínimo 18 anos. Veja o método que faz essa validação:

java 

Suponha agora que eu tenha outra classe, a classe `Produto`, que contém um atributo nome e eu quero fazer a mesma validação que fiz para o nome do usuário: Ver se só contém letras. E aí? Vou


# Tokens

Os **tokens** são uma sequência de caracteres (podem conter letras, números, pontuações, etc) separados por um limitador, que pode ser tanto um espaço em branco, como uma quebra de linha ou determinada pontuação).

**Tozenização** é um processo que divide strings em partes menores.

In [5]:
tokens = artigos.split()
print(f'Quantidade de tokens: {len(tokens)}')

Quantidade de tokens: 416903


In [6]:
tokens_2 = nltk.tokenize.word_tokenize(artigos)

In [7]:
print(tokens_2[:50])

['imagem', 'Temos', 'a', 'seguinte', 'classe', 'que', 'representa', 'um', 'usuário', 'no', 'nosso', 'sistema', ':', 'java', 'Para', 'salvar', 'um', 'novo', 'usuário', ',', 'várias', 'validações', 'são', 'feitas', ',', 'como', 'por', 'exemplo', ':', 'Ver', 'se', 'o', 'nome', 'só', 'contém', 'letras', ',', '[', '**o', 'CPF', 'só', 'números**', ']', 'e', 'ver', 'se', 'o', 'usuário', 'possui', 'no']


In [8]:
def separa_palavras(lista_tokens):
  lista_palavras = []

  for item in lista_tokens:
    if item.isalpha():
      lista_palavras.append(item)

  return lista_palavras    

palavras = separa_palavras(tokens_2)
print(f'Total de palavras: {len(palavras)}')

Total de palavras: 393914


# Normalização e Tipos de Palavras

In [9]:
def normaliza_palavras(lista):
  lista_normalizada = []

  for item in lista:
    lista_normalizada.append(item.lower())

  return lista_normalizada

lista_normalizada = normaliza_palavras(palavras)
print(lista_normalizada[:10])

['imagem', 'temos', 'a', 'seguinte', 'classe', 'que', 'representa', 'um', 'usuário', 'no']


In [10]:
total_palavras = set(lista_normalizada)
print(f'Total de palavras diferentes: {len(total_palavras)}.')

Total de palavras diferentes: 17652.


# Desenvolvendo o Corretor

In [16]:
palavra_exemplo = 'lgica'

def insere_palavras(letras):

  novas_palavras = []
  conjunto_letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'

  for left, right in letras:
    for l in conjunto_letras:
      novas_palavras.append(left + l + right)
  
  return novas_palavras


def gerador_palavras(palavra):

  letras = []

  for i in range(len(palavra) + 1):
    letras.append(((palavra[:i]), (palavra[i:])))

  palavras_geradas = insere_palavras(letras)
  return palavras_geradas

n_palavras = gerador_palavras(palavra_exemplo)
print(n_palavras)

['algica', 'blgica', 'clgica', 'dlgica', 'elgica', 'flgica', 'glgica', 'hlgica', 'ilgica', 'jlgica', 'klgica', 'llgica', 'mlgica', 'nlgica', 'olgica', 'plgica', 'qlgica', 'rlgica', 'slgica', 'tlgica', 'ulgica', 'vlgica', 'wlgica', 'xlgica', 'ylgica', 'zlgica', 'àlgica', 'álgica', 'âlgica', 'ãlgica', 'èlgica', 'élgica', 'êlgica', 'ìlgica', 'ílgica', 'îlgica', 'òlgica', 'ólgica', 'ôlgica', 'õlgica', 'ùlgica', 'úlgica', 'ûlgica', 'çlgica', 'lagica', 'lbgica', 'lcgica', 'ldgica', 'legica', 'lfgica', 'lggica', 'lhgica', 'ligica', 'ljgica', 'lkgica', 'llgica', 'lmgica', 'lngica', 'logica', 'lpgica', 'lqgica', 'lrgica', 'lsgica', 'ltgica', 'lugica', 'lvgica', 'lwgica', 'lxgica', 'lygica', 'lzgica', 'làgica', 'lágica', 'lâgica', 'lãgica', 'lègica', 'légica', 'lêgica', 'lìgica', 'lígica', 'lîgica', 'lògica', 'lógica', 'lôgica', 'lõgica', 'lùgica', 'lúgica', 'lûgica', 'lçgica', 'lgaica', 'lgbica', 'lgcica', 'lgdica', 'lgeica', 'lgfica', 'lggica', 'lghica', 'lgiica', 'lgjica', 'lgkica', 'lglica',